# Run Scores II for a Volume carrier

In [1]:
# %load imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import OrderedDict
#plt.style.use('paper')

#import data
import copy
from mdldb.run import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

from sklearn.metrics import r2_score
from src.data import database
from mdldb import tables
import shipflowmotionshelpers.shipflowmotionshelpers as helpers


Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 461 ('figure.figsize   : 5, 3   ## figure size in inches')
Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 462 ('figure.dpi       : 100        ## figure dots per inch')


In [2]:
from pyscores2.indata import Indata
from pyscores2.runScores2 import Calculation
from pyscores2.output import OutputFile
from pyscores2 import TDPError
import pyscores2
from rolldecayestimators.ikeda import Ikeda, IkedaR
from rolldecayestimators.simplified_ikeda_class import SimplifiedIkeda
import subprocess
from rolldecayestimators import measure
from src import run_scores

In [3]:
df_sections = pd.read_csv('../data/interim/volume_carrier_areas.csv', sep=';')
#volume_correction = 1.005
#df_sections['area']*=volume_correction
df_sections.head()

,no,area,x,t,b,r_b
0,0,3.152037,3.555946,0.32,15.590334,1.529306
1,1,80.056100,13.298716,6.77,40.829136,15.811695
2,2,186.996780,23.041486,7.50,42.999309,13.134742
3,3,263.957911,31.392432,7.50,43.000004,8.633547
4,4,310.254925,41.135203,7.50,43.000004,3.948538


In [49]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.99, 
                             exclude_table_name='rolldecay_exclude')

In [50]:
mask = df_rolldecay['loading_condition_id'] == 134
df = df_rolldecay.loc[mask]

In [51]:
df_sections['loading_condition_id'] = 134
section_groups = df_sections.groupby(by='loading_condition_id')

In [52]:
df['kg']

run_id
18405    19.09
18407    19.09
Name: kg, dtype: float64

In [53]:
df.KXX

run_id
18405    15.19
18407    15.19
Name: KXX, dtype: float64

In [54]:
df.KZZ

run_id
18405    57.57
18407    57.57
Name: KZZ, dtype: float64

In [55]:
df.gm

run_id
18405    8.02
18407    8.02
Name: gm, dtype: float64

In [56]:
results = pd.DataFrame()
fi_as = np.deg2rad([1,10])
for run_name, row in df.iterrows():
    
    loading_condition_id=row['loading_condition_id']
    sections = section_groups.get_group(loading_condition_id)
    
    sections=run_scores.remove_duplicate_sections(sections)
    sections=run_scores.too_small_sections(sections)
    
    try:
        indata,sections_ = run_scores.define_indata(row, sections, displacement_correction=1.01, lcb_correction=0.99)
    except run_scores.DraughtError as e:
        print('Draught is too large for sections, this loading condition is skipped.')
        continue

        
    save_name='%s.in' % row.loading_condition_id
    save_path=os.path.join('scores',save_name)
    indata.save(save_path)
    
    calculation = Calculation(outDataDirectory='scores/result')
    
    # Run scoresII:
    try:
        calculation.run(indata=indata, b_div_t_max=None, timeout=1.0)
    except TDPError:
        print('Dissregarding the TDPError')
        continue
    except pyscores2.LcgError as e:
        print('Disregarded')
        print(e)
        continue
    except subprocess.TimeoutExpired:
        print('Disregarded, scoresII got stuck...')
        continue
        
    output_file = OutputFile(filePath=calculation.outDataPath)   

Running Scores2 for loading_condition_id_134
Disregarded, scoresII got stuck...
Running Scores2 for loading_condition_id_134
Disregarded, scoresII got stuck...


In [48]:
row.Volume

49410.0

In [22]:
indata.speedMax

10.0

In [24]:
indata.waveFrequenciesMin

0.23564899342081083

In [25]:
indata.waveFrequenciesIncrement

0.017673674506560814

In [26]:
row.scale_factor

42.0

In [27]:
row.omega0

3.05436004440219